# Imports

In [1]:
import gspread_pandas
import pandas as pd

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
import logging
import helpers.hdbg as hdbg
import linkedin.google_api.google_file_api as google_file_api

In [3]:
_LOG = logging.getLogger(__name__)
hdbg.init_logger(use_exec_path=True)

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-10a5d1e3-db2a-4cc8-9431-8c69158af31c.json'
INFO: Saving log to file '/app/linkedin/notebooks/none.log'


# Load data

In [12]:
spreadsheet_name = "3_profile_export_sns5"
spread = gspread_pandas.Spread(spreadsheet_name)
df = spread.sheet_to_df(index=None)
print(df.shape)
df.head()

(5, 64)


,error,baseUrl,timestamp,linkedinProfileUrl,email,linkedinProfile,description,headline,location,imgUrl,...,skill6,endorsement6,profileId,mail,companyUrl,companyUrl2,schoolUrl2,website,schoolDateRange,schoolDateRange2
0,Not a LinkedIn Profile URL,profileUrl,2023-07-19T19:21:54.310Z,,,,,,,,...,,,,,,,,,,
1,,https://www.linkedin.com/sales/lead/ACwAAAHVj4...,2023-07-19T19:22:11.128Z,https://www.linkedin.com/in/maureenbcthomas/,,https://www.linkedin.com/in/maureenbcthomas/,"A visionary strategist, entrepreneur and innov...",Stakeholder Engagement I Innovative Coalition ...,Washington DC-Baltimore Area,https://media.licdn.com/dms/image/C5603AQGVCII...,...,Coaching & Mentoring,1,maureenbcthomas,,,,,,,
2,,https://www.linkedin.com/sales/lead/ACwAABgNPU...,2023-07-19T19:22:32.106Z,https://www.linkedin.com/in/edgarnoumair/,ednoumair@hotmail.com,https://www.linkedin.com/in/edgarnoumair/,I Advise & Lead Boards & Organizations on Acti...,Data Scientist / Entrepreneur / Author / Artis...,Washington DC-Baltimore Area,https://media.licdn.com/dms/image/C5603AQFNGxE...,...,Grant Writing,4,edgarnoumair,ednoumair@hotmail.com,https://www.linkedin.com/company/10522559/,https://www.linkedin.com/company/4794/,https://www.linkedin.com/company/3165/,gnoci.com,,
3,,https://www.linkedin.com/sales/lead/ACwAAAE1Yq...,2023-07-19T19:22:51.105Z,https://www.linkedin.com/in/nrdavis/,,https://www.linkedin.com/in/nrdavis/,,Assisting Maryland's startup companies and ent...,"Towson, Maryland, United States",https://media.licdn.com/dms/image/D5603AQGPI--...,...,New Business Development,22,nrdavis,,https://www.linkedin.com/company/5337426/,https://www.linkedin.com/company/1734721/,https://www.linkedin.com/company/19308/,,1979 - 1982,1971 - 1975
4,,https://www.linkedin.com/sales/lead/ACwAAACRHS...,2023-07-19T19:23:08.967Z,https://www.linkedin.com/in/discoveringmind/,,https://www.linkedin.com/in/discoveringmind/,I help corporate and real estate teams plan an...,"Finance/M&A Entrepreneur, Director & Advisor. ...","Annapolis, Maryland, United States",https://media.licdn.com/dms/image/C4D03AQF-DLN...,...,Investment Banking,35,discoveringmind,,,https://www.linkedin.com/company/59839/,https://www.linkedin.com/company/4477/,,1986 - 1988,Jul 2021 - Aug 2021


In [13]:
df.columns

Index(['error', 'baseUrl', 'timestamp', 'linkedinProfileUrl', 'email',
       'linkedinProfile', 'description', 'headline', 'location', 'imgUrl',
       'firstName', 'lastName', 'fullName', 'subscribers', 'connectionDegree',
       'vmid', 'userId', 'linkedinSalesNavigatorUrl', 'connectionsCount',
       'connectionsUrl', 'mutualConnectionsUrl', 'mutualConnectionsText',
       'company', 'jobTitle', 'jobDescription', 'jobLocation', 'jobDateRange',
       'jobDuration', 'company2', 'jobTitle2', 'jobDescription2',
       'jobLocation2', 'jobDateRange2', 'jobDuration2', 'school', 'schoolUrl',
       'schoolDegree', 'schoolDescription', 'school2', 'schoolDegree2',
       'schoolDescription2', 'twitter', 'twitterProfileUrl', 'allSkills',
       'skill1', 'endorsement1', 'skill2', 'endorsement2', 'skill3',
       'endorsement3', 'skill4', 'endorsement4', 'skill5', 'endorsement5',
       'skill6', 'endorsement6', 'profileId', 'mail', 'companyUrl',
       'companyUrl2', 'schoolUrl2', 'website'

In [14]:
for col in df.columns:
    print(col)
    print(df.iloc[0][col])
    print()

error
Not a LinkedIn Profile URL

baseUrl
profileUrl

timestamp
2023-07-19T19:21:54.310Z

linkedinProfileUrl


email


linkedinProfile


description


headline


location


imgUrl


firstName


lastName


fullName


subscribers


connectionDegree


vmid


userId


linkedinSalesNavigatorUrl


connectionsCount


connectionsUrl


mutualConnectionsUrl


mutualConnectionsText


company


jobTitle


jobDescription


jobLocation


jobDateRange


jobDuration


company2


jobTitle2


jobDescription2


jobLocation2


jobDateRange2


jobDuration2


school


schoolUrl


schoolDegree


schoolDescription


school2


schoolDegree2


schoolDescription2


twitter


twitterProfileUrl


allSkills


skill1


endorsement1


skill2


endorsement2


skill3


endorsement3


skill4


endorsement4


skill5


endorsement5


skill6


endorsement6


profileId


mail


companyUrl


companyUrl2


schoolUrl2


website


schoolDateRange


schoolDateRange2




# Filter data

In [15]:
# Drop errors.
prev_len = len(df)
df = df[df["error"] == ""].reset_index(drop=True)
df = df[[col for col in df.columns if col != "error"]]
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)
df.head()

Dropped 1 rows (20.0%)


,baseUrl,timestamp,linkedinProfileUrl,email,linkedinProfile,description,headline,location,imgUrl,firstName,...,skill6,endorsement6,profileId,mail,companyUrl,companyUrl2,schoolUrl2,website,schoolDateRange,schoolDateRange2
0,https://www.linkedin.com/sales/lead/ACwAAAHVj4...,2023-07-19T19:22:11.128Z,https://www.linkedin.com/in/maureenbcthomas/,,https://www.linkedin.com/in/maureenbcthomas/,"A visionary strategist, entrepreneur and innov...",Stakeholder Engagement I Innovative Coalition ...,Washington DC-Baltimore Area,https://media.licdn.com/dms/image/C5603AQGVCII...,Maureen,...,Coaching & Mentoring,1,maureenbcthomas,,,,,,,
1,https://www.linkedin.com/sales/lead/ACwAABgNPU...,2023-07-19T19:22:32.106Z,https://www.linkedin.com/in/edgarnoumair/,ednoumair@hotmail.com,https://www.linkedin.com/in/edgarnoumair/,I Advise & Lead Boards & Organizations on Acti...,Data Scientist / Entrepreneur / Author / Artis...,Washington DC-Baltimore Area,https://media.licdn.com/dms/image/C5603AQFNGxE...,Dr. Edgar,...,Grant Writing,4,edgarnoumair,ednoumair@hotmail.com,https://www.linkedin.com/company/10522559/,https://www.linkedin.com/company/4794/,https://www.linkedin.com/company/3165/,gnoci.com,,
2,https://www.linkedin.com/sales/lead/ACwAAAE1Yq...,2023-07-19T19:22:51.105Z,https://www.linkedin.com/in/nrdavis/,,https://www.linkedin.com/in/nrdavis/,,Assisting Maryland's startup companies and ent...,"Towson, Maryland, United States",https://media.licdn.com/dms/image/D5603AQGPI--...,Neil,...,New Business Development,22,nrdavis,,https://www.linkedin.com/company/5337426/,https://www.linkedin.com/company/1734721/,https://www.linkedin.com/company/19308/,,1979 - 1982,1971 - 1975
3,https://www.linkedin.com/sales/lead/ACwAAACRHS...,2023-07-19T19:23:08.967Z,https://www.linkedin.com/in/discoveringmind/,,https://www.linkedin.com/in/discoveringmind/,I help corporate and real estate teams plan an...,"Finance/M&A Entrepreneur, Director & Advisor. ...","Annapolis, Maryland, United States",https://media.licdn.com/dms/image/C4D03AQF-DLN...,Michael,...,Investment Banking,35,discoveringmind,,,https://www.linkedin.com/company/59839/,https://www.linkedin.com/company/4477/,,1986 - 1988,Jul 2021 - Aug 2021


In [16]:
# Filter by keywords.
keywords = ["volunteer", "adjunct", "consult"]
prev_len = len(df)
dfs_filtered = []
for col in [
    "headline",
    "jobTitle",
    "jobDescription",
    "jobTitle2",
    "jobDescription2",
    "allSkills",
]:
    dfs_filtered.append(
        df[df[col].apply(lambda x: any(kw in x.lower() for kw in keywords))]
    )
df = pd.concat(dfs_filtered).drop_duplicates().reset_index(drop=True)
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)

Dropped 1 rows (25.0%)


In [17]:
print(df.shape)
df

(3, 63)


,baseUrl,timestamp,linkedinProfileUrl,email,linkedinProfile,description,headline,location,imgUrl,firstName,...,skill6,endorsement6,profileId,mail,companyUrl,companyUrl2,schoolUrl2,website,schoolDateRange,schoolDateRange2
0,https://www.linkedin.com/sales/lead/ACwAAAHVj4...,2023-07-19T19:22:11.128Z,https://www.linkedin.com/in/maureenbcthomas/,,https://www.linkedin.com/in/maureenbcthomas/,"A visionary strategist, entrepreneur and innov...",Stakeholder Engagement I Innovative Coalition ...,Washington DC-Baltimore Area,https://media.licdn.com/dms/image/C5603AQGVCII...,Maureen,...,Coaching & Mentoring,1,maureenbcthomas,,,,,,,
1,https://www.linkedin.com/sales/lead/ACwAABgNPU...,2023-07-19T19:22:32.106Z,https://www.linkedin.com/in/edgarnoumair/,ednoumair@hotmail.com,https://www.linkedin.com/in/edgarnoumair/,I Advise & Lead Boards & Organizations on Acti...,Data Scientist / Entrepreneur / Author / Artis...,Washington DC-Baltimore Area,https://media.licdn.com/dms/image/C5603AQFNGxE...,Dr. Edgar,...,Grant Writing,4,edgarnoumair,ednoumair@hotmail.com,https://www.linkedin.com/company/10522559/,https://www.linkedin.com/company/4794/,https://www.linkedin.com/company/3165/,gnoci.com,,
2,https://www.linkedin.com/sales/lead/ACwAAAE1Yq...,2023-07-19T19:22:51.105Z,https://www.linkedin.com/in/nrdavis/,,https://www.linkedin.com/in/nrdavis/,,Assisting Maryland's startup companies and ent...,"Towson, Maryland, United States",https://media.licdn.com/dms/image/D5603AQGPI--...,Neil,...,New Business Development,22,nrdavis,,https://www.linkedin.com/company/5337426/,https://www.linkedin.com/company/1734721/,https://www.linkedin.com/company/19308/,,1979 - 1982,1971 - 1975


# Save filtered data to gsheet

## Create a empty Google sheet

In [18]:
gapi = google_file_api.GoogleFileApi()

INFO  file_cache is only supported with oauth2client<4.0.0


In [19]:
name = 'SN_Search5_Yiyun'
# gdrive_folder : dict, the id and the name of the Google Drive folder.
gdrive_folder  = gapi.get_folder_id_by_name(name)

INFO  Found folder: {'id': '1XWNGDnJrVICHAe-6V2cnoSklZpk0APc_', 'name': 'SN_Search5_Yiyun'}


In [6]:
# if you want to use another folder id, please change the folder id manually.
# gdrive_folder  = {'id': '1XWNGDnJrVICHAe-6V2cnoSklZpk0APc_', 'name': 'SN_Search5_Yiyun'} 

In [8]:
"""
Create a new Google file (sheet or doc).

:param gfile_type: str, the type of the Google file ('sheet' or 'doc').
:param gfile_name: str, the name of the new Google file.
:param folder_id: str, the id of the Google Drive folder.
:param user: str, the email address of the user to share the Google file (Optional).
:return: None
"""
gfile_type = 'sheet'
gsheet_name = '4_profile_export_filtered_sns5'
user = ''

In [10]:
gapi.create_empty_google_file(
    gfile_type = gfile_type,
    gfile_name = gsheet_name,
    gdrive_folder = gdrive_folder, 
    user = user
)

INFO  file_cache is only supported with oauth2client<4.0.0
INFO  Created a new Google sheet 'sn_search5.profile_export.filtered.gsheet'.
INFO  Move the new Google sheet 'sn_search5.profile_export.filtered.gsheet' to the dir 'SN_Search5_Yiyun'
INFO  Finished creating the new Google sheet 'sn_search5.profile_export.filtered.gsheet'.


## Save filtered data

In [20]:
# A Google sheet with this name should already exist on the drive.
spread2 = gspread_pandas.Spread(
    gsheet_name,
    sheet="Sheet1",
    create_sheet=True,
)
spread2.df_to_sheet(df, index=False)